In [ ]:
import datetime as dt
import pandas as pd

from gs_quant.markets.position_set import Position, PositionSet
from gs_quant.session import Environment, GsSession

In [ ]:
client = 'CLIENT ID'
secret = 'CLIENT SECRET'

GsSession.use(Environment.PROD, client_id=client, client_secret=secret, scopes=('read_product_data',))

If you have historical positions that you'd like to bring into Marquee, you should create an individual position
set per each position date. The below examples show the three valid ways to do this (column names/format included)

### Step 1: Define your file format
#### Option 1: Create historical position sets by weight
<table>
    <th>identifier</th><th>weight</th><th>date</th>
    <tr>
        <td>AAPL UW</td>
        <td>0.4</td>
        <td>2022-06-03</td>
    </tr>
    <tr>
        <td>MSFT UW</td>
        <td>0.6</td>
        <td>2022-06-03</td>
    </tr>
    <tr>
        <td>AAPL UW</td>
        <td>0.4</td>
        <td>2023-01-04</td>
    </tr>
    <tr>
        <td>MSFT UW</td>
        <td>0.2</td>
        <td>2023-01-04</td>
    </tr>
    <tr>
        <td>IBM UN</td>
        <td>0.4</td>
        <td>2023-01-04</td>
    </tr>
</table>

In [ ]:
columns = ['identifier', 'weight', 'date']
equalize = False

#### Option 2: Create historical position sets by quantity

<table>
    <th>identifier</th><th>quantity</th><th>date</th>
    <tr>
        <td>AAPL UW</td>
        <td>100</td>
        <td>2022-06-03</td>
    </tr>
    <tr>
        <td>MSFT UW</td>
        <td>100</td>
        <td>2022-06-03</td>
    </tr>
    <tr>
        <td>AAPL UW</td>
        <td>200</td>
        <td>2023-01-04</td>
    </tr>
    <tr>
        <td>MSFT UW</td>
        <td>300</td>
        <td>2023-01-04</td>
    </tr>
    <tr>
        <td>IBM UN</td>
        <td>200</td>
        <td>2023-01-04</td>
    </tr>
</table>

In [ ]:
columns = ['identifier', 'quantity', 'date']
equalize = False

#### Option 3: Create equally-weighted historical position sets
<table>
    <th>identifier</th><th>date</th>
    <tr>
        <td>AAPL UW</td>
        <td>2022-06-03</td>
    </tr>
    <tr>
        <td>MSFT UW</td>
        <td>2022-06-03</td>
    </tr>
    <tr>
        <td>AAPL UW</td>
        <td>2023-01-04</td>
    </tr>
    <tr>
        <td>MSFT UW</td>
        <td>2023-01-04</td>
    </tr>
    <tr>
        <td>IBM UN</td>
        <td>2023-01-04</td>
    </tr>
</table>

In [ ]:
columns = ['identifier', 'date']
equalize = True

### Step 2: Upload your excel file and convert your positions into position sets

In [ ]:
positions_data = pd.read_excel('positions_data.xlsx', engine='openpyxl')
all_positions = positions_data[columns]

positions_dict, position_sets = {}, []
for i, row in all_positions.iterrows():
    date = row.get('date')
    if not date in positions_dict:
        positions_dict[date] = set()
    positions_dict[date].add(Position(identifier=row.get('identifier'),
                                      weight=row.get('weight', None),
                                      quantity=row.get('quantity', None)))

for date in positions_dict:
    position_date = dt.datetime.strptime(str(date), '%Y-%m-%d %H:%M:%S').date()
    position_set = PositionSet(positions=positions_dict[date], date=position_date)
    if equalize:
        position_set.equalize_position_weights()
    position_sets.append(position_set)

print(position_sets)